# kmeans autocropper

Need to:
1. autocrop images
1. save as uncompressed, 8-bit TIFF tagged with AdobeRGB

In [2]:
%matplotlib inline
from pathlib import Path

import cv2
import numpy as np
from ipywidgets import IntProgress, Label, VBox
from IPython.display import display
from matplotlib import pyplot as plt
from PIL import Image
from sklearn.cluster import MiniBatchKMeans

import img_qc.img_qc as img_qc

In [91]:
def rotate_bound(image, angle, center=None, scale=1.0):
    # grab the dimensions of the image and then determine the
    # center
    (height, width) = image.shape[:2]
    
    # if the center is None, initialize it as the center of
    # the image
    if center is None:
        centerX = (w // 2)
        centerY = (h // 2)
    else:
        centerX, centerY = center

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((centerX, centerY), -angle, scale)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    width_new = int((height * sin) + (width * cos))
    height_new = int((height * cos) + (width * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (width_new / 2) - centerX
    M[1, 2] += (height_new / 2) - centerY

    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (width_new, height_new), flags=cv2.INTER_CUBIC)
    

def autocrop(image_path, compression=None, dpi=None, padding=0):
    
    # set debug directory
    debug_directory_path = image_path.parents[0].joinpath('kmeans')
    debug_directory_path.mkdir(exist_ok=True)
    
    # === AutoCrop
    if not dpi:
        image = Image.open(image_path)
        dpi = image.info['dpi'][0]

    # load the image
    image_original = cv2.imread(str(image_path))
    
    # compute the ratio of the max height of the sensor to the processed image
    ratio = 6192 / image_original.shape[0]  # height of Fuji GFX50s sensor in pixels
    
    # load kmeans image
    kmeans_name = f'{image_path.stem}_kmeans.png'
    kmeans_image_path = data_path.joinpath('kmeans', kmeans_name)
    kmeans_image = cv2.imread(str(kmeans_image_path), cv2.IMREAD_GRAYSCALE)
    
    # convert the image to grayscale
    #gray = cv2.cvtColor(kmeans_image, cv2.COLOR_BGR2GRAY)
    
    # find the contours in the thresholded image keeping the external one
    _, contours, hierarchy = cv2.findContours(kmeans_image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    cnts = contours
    
    # sort the contours from left to right
    (cnts, bounding_boxes) = img_qc.sort_contours(cnts)
    
    # loop over the contours individually
    for (i, c) in enumerate(cnts):
        # if the contour is not sufficiently large, ignore it
        if cv2.contourArea(c) < 500000:  # use 20000 for scrapbook pages
            continue
            
        # compute the rotated bounding box of the contour
        box = cv2.minAreaRect(c)
        box = cv2.boxPoints(box)
        box = np.int0(box)
        
        # DEBUG: draw found contour & show image
        clone = image_original.copy()
        cv2.drawContours(clone, [box], 0, (0, 0, 255), 2)
        pil_image = Image.fromarray(clone)
        contour_name = f'{image_path.stem}_contour_{str(i).zfill(3)}{image_path.suffix}'
        contour_jpg_path = debug_directory_path.joinpath(contour_name)
        pil_image.save(contour_jpg_path)
        
        # re-order the points in tl, tr, br, bl order
        rect = img_qc.order_points(box)
        
        # find the points and angle for minAreaRectangle
        (x, y), (w, h), theta = cv2.minAreaRect(c)
        
        # rotate image around center of minAreaRect by theta amount
        #if theta < -45:
            #theta = 90 + theta
            
        # the `cv2.minAreaRect` function returns values in the
        # range [-90, 0); as the rectangle rotates clockwise the
        # returned angle trends to 0 -- in this special case we
        # need to add 90 degrees to the angle
        if theta < -45:
            angle = -(90 + theta)
 
        # otherwise, just take the inverse of the angle to make
        # it positive
        else:
            angle = -theta
    
    # multiply the rectangle by the original ratio
    #rect *= ratio
    
    # find the points we need to crop the full size original
    tl, tr, br, bl = rect
    startX = max(min(tl[0], bl[0]), 0)
    startY = max(min(tl[1], tr[1]), 0)
    endX = max(tr[0], br[0])
    endY = max(bl[1], br[1])
    
    # rotate original by theta from minAreaRect
    #x *= ratio
    #y *= ratio
    image_rotated = img_qc.rotate(image_original.copy(), -angle, (x, y))
    
    # add padding (default hard-coded is 0 pixels)
    pixel_padding = int(padding)
    startX -= pixel_padding
    startY -= pixel_padding
    endX += pixel_padding
    endY += pixel_padding
    
    # startX/startY to max of current value and 0 to stay inside image
    startX = max(startX, 0)
    startY = max(startY, 0)
    
    # endX/endY to min of current value and max width/height of image to stay inside image
    endX = min(endX, 8256)  # NOTE: NOT USING ROTATED MAX SIZE
    endY = min(endY, 6192)
    
    # crop the image in memory
    image_cropped = image_rotated[int(startY):int(endY), int(startX):int(endX)]
    
    # create output directory and set output path
    output_directory_path = image_path.parents[0].joinpath('00_cropped')
    output_directory_path.mkdir(exist_ok=True)
    output_path = output_directory_path.joinpath(image_path.name)
    
    # convert to pillow Image
    #image_test = cv2.cvtColor(image_cropped, cv2.COLOR_BGR2RGB)  # convert to RGB!
    pillow_image = Image.fromarray(image_cropped)
    
    dpi = float(dpi)  # dpi MUST be a float for Pillow
        
    pillow_image.save(output_path, compression=compression, dpi=(dpi, dpi))
    
    crop_box = [int(startY), int(endY), int(startX), int(endX)]
    
    # round off theta to 3 digits
    # rotation_angle = round(angle, 3)
    rotation_angle = angle
    
    # get angle of rotation and 
    if rotation_angle < 0:
        rotation_direction = 'ccw'
    else:
        rotation_direction = 'cw'
    
    # rotation, centerX, centerY, width, height
    capture_one_data = [rotation_direction, rotation_angle, int(x), int(y), int(endX - startX), int(endY - startY)]
    
    capture_one_data_as_str = [str(x) for x in capture_one_data]
    
    return capture_one_data_as_str

In [92]:
data_path = Path('/Users/jeremy/Pictures/bennett_pack-film/Output/_autocrop_jpg/')

image_paths_list = sorted(data_path.glob('*.jpg'))
len(image_paths_list)

17

In [93]:
for image_path in image_paths_list[:-3]:
    print(image_path.stem)
    capture_one_data_as_str = autocrop(image_path, padding=10)
    print("\n    ".join(capture_one_data_as_str))

MS3892-B2-S11-F35_001
cw
    0.5122091770172119
    535
    389
    970
    587
MS3892-B2-S11-F35_002
cw
    0.14377830922603607
    536
    304
    954
    581
MS3892-B2-S11-F35_003
cw
    0.15360763669013977
    508
    340
    956
    581
MS3892-B2-S11-F35_004
ccw
    -0.0657806396484375
    517
    373
    946
    588
MS3892-B2-S11-F35_005
cw
    0.20809513330459595
    517
    317
    958
    589
MS3892-B2-S11-F35_006
cw
    0.1968919336795807
    524
    369
    978
    590
MS3892-B2-S11-F35_007
ccw
    -0.270263671875
    517
    342
    970
    592
MS3892-B2-S11-F35_008
cw
    0.19917342066764832
    522
    367
    984
    593
MS3892-B2-S11-F35_009
cw
    0.280172735452652
    520
    310
    998
    592
MS3892-B2-S11-F35_010
cw
    0.24175290763378143
    522
    304
    976
    591
MS3892-B2-S11-F35_011
cw
    0.12482722103595734
    532
    387
    1043
    589
MS3892-B2-S11-F35_012
cw
    0.0
    533
    338
    943
    587
MS3892-B2-S11-F35_013
cw
    0.40253981947898865


In [94]:
def rotate_bound(image, angle, center=None, scale=1.0):
    # grab the dimensions of the image and then determine the
    # center
    (height, width) = image.shape[:2]
    
    # if the center is None, initialize it as the center of
    # the image
    if center is None:
        centerX = (w // 2)
        centerY = (h // 2)
    else:
        centerX, centerY = center

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((centerX, centerY), -angle, scale)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    width_new = int((height * sin) + (width * cos))
    height_new = int((height * cos) + (width * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (width_new / 2) - centerX
    M[1, 2] += (height_new / 2) - centerY

    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (width_new, height_new), flags=cv2.INTER_CUBIC)
    

def autocrop(image_path, compression=None, dpi=None, padding=0):
    
    # set debug directory
    debug_directory_path = image_path.parents[0].joinpath('otsu')
    debug_directory_path.mkdir(exist_ok=True)
    
    # === AutoCrop
    if not dpi:
        image = Image.open(image_path)
        dpi = image.info['dpi'][0]

    # load the image
    image_original = cv2.imread(str(image_path))
    
    # compute the ratio of the max height of the sensor to the processed image
    ratio = 6192 / image_original.shape[0]  # height of Fuji GFX50s sensor in pixels
    
    # load otsu image
    otsu_name = f'{image_path.stem}_otsu.png'
    otsu_image_path = data_path.joinpath('otsu', otsu_name)
    otsu_image = cv2.imread(str(otsu_image_path), cv2.IMREAD_GRAYSCALE)
    
    # convert the image to grayscale
    #gray = cv2.cvtColor(otsu_image, cv2.COLOR_BGR2GRAY)
    
    # find the contours in the thresholded image keeping the external one
    _, contours, hierarchy = cv2.findContours(otsu_image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    cnts = contours
    
    # sort the contours from left to right
    (cnts, bounding_boxes) = img_qc.sort_contours(cnts)
    
    # loop over the contours individually
    for (i, c) in enumerate(cnts):
        # if the contour is not sufficiently large, ignore it
        if cv2.contourArea(c) < 500000:  # use 20000 for scrapbook pages
            continue
            
        # compute the rotated bounding box of the contour
        box = cv2.minAreaRect(c)
        box = cv2.boxPoints(box)
        box = np.int0(box)
        
        # DEBUG: draw found contour & show image
        clone = image_original.copy()
        cv2.drawContours(clone, [box], 0, (0, 0, 255), 2)
        pil_image = Image.fromarray(clone)
        contour_name = f'{image_path.stem}_contour_{str(i).zfill(3)}{image_path.suffix}'
        contour_jpg_path = debug_directory_path.joinpath(contour_name)
        pil_image.save(contour_jpg_path)
        
        # re-order the points in tl, tr, br, bl order
        rect = img_qc.order_points(box)
        
        # find the points and angle for minAreaRectangle
        (x, y), (w, h), theta = cv2.minAreaRect(c)
        
        # rotate image around center of minAreaRect by theta amount
        #if theta < -45:
            #theta = 90 + theta
            
        # the `cv2.minAreaRect` function returns values in the
        # range [-90, 0); as the rectangle rotates clockwise the
        # returned angle trends to 0 -- in this special case we
        # need to add 90 degrees to the angle
        if theta < -45:
            angle = -(90 + theta)
 
        # otherwise, just take the inverse of the angle to make
        # it positive
        else:
            angle = -theta
    
    # multiply the rectangle by the original ratio
    #rect *= ratio
    
    # find the points we need to crop the full size original
    tl, tr, br, bl = rect
    startX = max(min(tl[0], bl[0]), 0)
    startY = max(min(tl[1], tr[1]), 0)
    endX = max(tr[0], br[0])
    endY = max(bl[1], br[1])
    
    # rotate original by theta from minAreaRect
    #x *= ratio
    #y *= ratio
    image_rotated = img_qc.rotate(image_original.copy(), -angle, (x, y))
    
    # add padding (default hard-coded is 0 pixels)
    pixel_padding = int(padding)
    startX -= pixel_padding
    startY -= pixel_padding
    endX += pixel_padding
    endY += pixel_padding
    
    # startX/startY to max of current value and 0 to stay inside image
    startX = max(startX, 0)
    startY = max(startY, 0)
    
    # endX/endY to min of current value and max width/height of image to stay inside image
    endX = min(endX, 8256)  # NOTE: NOT USING ROTATED MAX SIZE
    endY = min(endY, 6192)
    
    # crop the image in memory
    image_cropped = image_rotated[int(startY):int(endY), int(startX):int(endX)]
    
    # create output directory and set output path
    output_directory_path = image_path.parents[0].joinpath('00_cropped')
    output_directory_path.mkdir(exist_ok=True)
    output_path = output_directory_path.joinpath(image_path.name)
    
    # convert to pillow Image
    #image_test = cv2.cvtColor(image_cropped, cv2.COLOR_BGR2RGB)  # convert to RGB!
    pillow_image = Image.fromarray(image_cropped)
    
    dpi = float(dpi)  # dpi MUST be a float for Pillow
        
    pillow_image.save(output_path, compression=compression, dpi=(dpi, dpi))
    
    crop_box = [int(startY), int(endY), int(startX), int(endX)]
    
    # round off theta to 3 digits
    # rotation_angle = round(angle, 3)
    rotation_angle = angle
    
    # get angle of rotation and 
    if rotation_angle < 0:
        rotation_direction = 'ccw'
    else:
        rotation_direction = 'cw'
    
    # rotation, centerX, centerY, width, height
    capture_one_data = [rotation_direction, rotation_angle, int(x), int(y), int(endX - startX), int(endY - startY)]
    
    capture_one_data_as_str = [str(x) for x in capture_one_data]
    
    return capture_one_data_as_str

In [96]:
for image_path in image_paths_list[:-3]:
    print(image_path.stem)
    try:
        capture_one_data_as_str = autocrop(image_path, padding=10)
    except UnboundLocalError:
        print("*** Can't Process ***")
    print("\n    ".join(capture_one_data_as_str))

MS3892-B2-S11-F35_001
cw
    0.5148391127586365
    535
    389
    970
    587
MS3892-B2-S11-F35_002
*** Can't Process ***
cw
    0.5148391127586365
    535
    389
    970
    587
MS3892-B2-S11-F35_003
*** Can't Process ***
cw
    0.5148391127586365
    535
    389
    970
    587
MS3892-B2-S11-F35_004
*** Can't Process ***
cw
    0.5148391127586365
    535
    389
    970
    587
MS3892-B2-S11-F35_005
*** Can't Process ***
cw
    0.5148391127586365
    535
    389
    970
    587
MS3892-B2-S11-F35_006
cw
    0.2507457733154297
    525
    370
    979
    591
MS3892-B2-S11-F35_007
*** Can't Process ***
cw
    0.2507457733154297
    525
    370
    979
    591
MS3892-B2-S11-F35_008
cw
    0.19917342066764832
    522
    367
    984
    593
MS3892-B2-S11-F35_009
cw
    0.2741403877735138
    521
    310
    997
    592
MS3892-B2-S11-F35_010
*** Can't Process ***
cw
    0.2741403877735138
    521
    310
    997
    592
MS3892-B2-S11-F35_011
cw
    0.12482719868421555
    532
    387
  